Install Dependencies and Import Required Packages

In [1]:
# !sudo add-apt-repository ppa:openjdk-r/ppa
#!sudo apt-get install openjdk-11-jdk
# To Install Oracke JDK varsion 8
# !sudo add-apt-repository ppa:webupd8team/java
# !sudo apt-get install oracle-java8-installer

[sudo] password for kankan: 


In [2]:
# !wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
# !tar xvzf spark-3.1.1-bin-hadoop3.2.tgz
!pip install pyspark
!pip install -q findspark
!pip install pyarrow
try:
  # %tensorflow_version only exists in Colab.
  !pip install  tf-estimator-nightly==2.8.0.dev2021122109
except Exception:
  pass

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyspark-3.3.2.tar.gz (281.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824021 sha256=f22da6ba3c22d2a73f5284668a32c3724af769bd0c974d323f79b23c4a8af4aa
  Stored in directory: /home/kankan/.cache/pip/wheels/89/d6/52/1178e354ba2207673484f0ccd7b2ded0ab6671ae5c1fc5b49a
Successfully built pyspark
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 950.4 kB/s eta 0:00:00m eta 0:00:010:01:01


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1"


In [4]:
from pyspark.sql.functions import col, pandas_udf, regexp_extract
import io

from tensorflow.keras.applications.imagenet_utils import decode_predictions
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pathlib
import findspark
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt 


findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

ModuleNotFoundError: No module named 'tensorflow'

Download the Public Dataset

In [ ]:
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)

print(data_dir)

Recursively Read all the Images from the Parent directory and create a DataFrame

In [ ]:
images = spark.read.format("binaryFile").option("recursiveFileLookup", "true").option("pathGlobFilter", "*.jpg").load(data_dir)
print(type(images))

In [ ]:
images.show(5)

In [ ]:
files=images.select('path').rdd.map(lambda x :x.path ).collect()
files[0:10]

In [ ]:

def extract_label(path_col):
  """Extract label from file path using built-in SQL functions."""
  return regexp_extract(path_col, "flower_photos/([^/]+)", 1)

def extract_size(content):
  """Extract image size from its raw content."""
  image = Image.open(io.BytesIO(content))
  return image.size

@pandas_udf("width: int, height: int")
def extract_size_udf(content_series):
  sizes = content_series.apply(extract_size)
  return pd.DataFrame(list(sizes))

df = images.select(
  col("path"),
  col("modificationTime"),
  extract_label(col("path")).alias("label"),
  extract_size_udf(col("content")).alias("size"),
  col("content"))


df.show(5)


In [ ]:
ima=images.select('content').rdd.map(lambda x: x.content).collect()
ima[0]

In [ ]:
imgg = Image.open(io.BytesIO(ima[0]))
imgg

In [ ]:
len(ima)

In [ ]:
class ImageNetDataset(Dataset):
  """
  Converts image contents into a PyTorch Dataset with standard ImageNet preprocessing.
  """
  def __init__(self, contents):
    self.contents = contents

  def __len__(self):
    
    return len(self.contents)

  def __getitem__(self, index):
    return self._preprocess(self.contents[index])

  def _preprocess(self, content):
    """
    Preprocesses the input image content using standard ImageNet normalization.
    
    See https://pytorch.org/docs/stable/torchvision/models.html.
    """
    image = Image.open(io.BytesIO(content))
    transform = transforms.Compose([
      transforms.Resize(256),                         ### Smaller lenght is convertd to 256 
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

In [ ]:

def imagenet_model_udf():
  """
  Wraps an ImageNet model into a Pandas UDF that makes predictions.
  
  You might consider the following customizations for your own use case:
    - Tune DataLoader's batch_size and num_workers for better performance.
    - Use GPU for acceleration.
    - Change prediction types.
  """

  model=models.mobilenet_v2(pretrained=True)
  model.eval()
  def predict(content_series_iter : pd.Series) -> pd.DataFrame:
    # model = model_fn() # Once per Map
    # model.eval()      # Once Per Map ,model file read from memory
    for content_series in content_series_iter:                                                    #Iterates overall all Images 
      dataset = ImageNetDataset(list(content_series))                                           
      loader = DataLoader(dataset, batch_size=64) 
      with torch.no_grad():
        for image_batch in loader:
          predictions = model(image_batch).numpy()                                                # Predictions for all 1000 classes of Mobilenetv2 Training Dataset
          predicted_labels = [x[0] for x in decode_predictions(predictions, top=1)]                       
          yield pd.DataFrame(predicted_labels)
    
        
  return_type = "class: string, desc: string, score:float"
  # return_type = "class: string"                                         # SCALAR_ITER implies the function accepts list/array of scalar values as input.
  return pandas_udf(predict,return_type, PandasUDFType.SCALAR_ITER)   
  # return pandas_udf(return_type, PandasUDFType.SCALAR_ITER)(predict)   
  
  #Returns a Predict function object as the final user defined function object.
  #Iterates over an Image: A sequence of bytes represented as a 2D array. 
  #Scalar is a series of bytes.    

In [ ]:
# imagenet = imagenet_model_udf(lambda: models.mobilenet_v2(pretrained=True))
imagenet = imagenet_model_udf()
imagenet

In [ ]:
predictions = df.withColumn("prediction", imagenet(col("content")))
display(predictions.select(col("path"), col("prediction")).limit(5))

In [ ]:
predictions.select(col("label"),col("prediction")).show(50,truncate=False)

Experimental Code Cells


In [ ]:
cont_series= list(df.select('content').rdd.map(lambda x: x.content).collect())
print(cont_series[0])

In [ ]:
# for  i in cont_series:

dataset = ImageNetDataset([cont_series[0]])             
loader = DataLoader(dataset, batch_size=64) 
  

In [ ]:
for i in loader:
  print(i.shape)
# t=iter(loader)
# import matplotlib.pyplot as plt 
# plt.figure()
# plt.imshow(first[0,0].numpy())


In [ ]:
model =models.mobilenet_v2(pretrained=True)
model.eval()

c=0
with torch.no_grad():
    for image_batch in loader:
      predictions = model(image_batch).numpy()
      predicted_labels = [x for x in decode_predictions(predictions, top=1)]  
      print(predicted_labels)
      c+=1
      if c==5:
        break


https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c 